In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.7 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel

class ExpressiveParaphraser(nn.Module):
    def __init__(self, model_name):
        super(ExpressiveParaphraser, self).__init__()
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.bert = BertModel.from_pretrained(model_name)
        self.signal_embedding = nn.Embedding(2, self.bert.config.hidden_size)  # Match the hidden size
        self.fc = nn.Linear(self.bert.config.hidden_size, self.tokenizer.vocab_size)

    def forward(self, input_ids, attention_mask, signal, token_type_ids=None):
        signal_emb = self.signal_embedding(signal).unsqueeze(1).repeat(1, input_ids.size(1), 1)  # Repeat for the sequence length
        input_emb = self.bert.embeddings(input_ids)

        # Combining the signal embedding with input embeddings
        combined_emb = input_emb + signal_emb

        # Passing combined embeddings through BERT
        outputs = self.bert(inputs_embeds=combined_emb, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state
        logits = self.fc(sequence_output)

        return logits


# If you have a GPU available on Colab
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "bert-base-uncased"
model = ExpressiveParaphraser(model_name).to(device)


In [ ]:
inputs = model.tokenizer(["I love apples."], return_tensors="pt", padding=True, truncation=True)
signal = torch.tensor([1]).to(device)  # for expressive paraphrase

inputs = {key: val.to(device) for key, val in inputs.items()}

with torch.no_grad():
    logits = model(**inputs, signal=signal)

predicted_token_ids = torch.argmax(logits, dim=-1)
predicted_text = model.tokenizer.decode(predicted_token_ids[0], skip_special_tokens=True)
print(predicted_text)

hundreds camps windows hundreds degraded north
